# Comprehensive Respondent-Level EDA for Wonky Study Detection

This notebook contains expanded EDA analysis on `respondent_features` dataframe to identify features that help distinguish users involved in wonky studies.

**Key Sections:**
- Comprehensive statistical testing on behavioral features
- Feature ranking and importance analysis
- Correlation and multicollinearity detection
- Interaction feature creation and testing
- Threshold analysis for optimal cutpoints
- Final feature list organized by tiers
- Modeling recommendations

---

Exec Summary

-
-
-
-
-

In [0]:
%pip install pyyaml>=6.0 -q

In [0]:
# Import libraries
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import numpy as np
import pandas as pd

try:
    import yaml
except ImportError:
    raise ImportError(
        "PyYAML is not installed. Please run the previous cell to install it, "
        "or run: %pip install pyyaml>=6.0"
    )

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from eda.feature_engineering import (
    create_time_features,
    create_task_speed_features,
    create_respondent_behavioral_features,
    add_wonky_features,
    create_fraud_risk_score,
    create_wonky_risk_score
)
from eda.statistical_tests import (
    compare_groups_statistically,
    compare_groups_with_both_tests,
    analyze_thresholds,
    perform_chi_square_tests,
    compare_demographic_groups
)
from eda.visualizations import (
    create_histogram,
    create_box_plot,
    create_scatter_plot,
    create_bar_plot,
    create_temporal_breakdown_summary,
    create_task_speed_breakdown_summary,
    create_chi_squared_bar_chart,
    create_dual_axis_statistical_chart,
    create_feature_breakdown_table,
    create_distribution_comparison,
    calculate_temporal_feature_deltas,       
    create_chi_squared_delta_dual_axis_chart,
)

# Load configs
with open('../configs/feature_engineering.yaml', 'r') as f:
    feature_config = yaml.safe_load(f)

with open('../configs/statistical_tests.yaml', 'r') as f:
    stats_config = yaml.safe_load(f)

with open('../configs/data_paths.yaml', 'r') as f:
    paths_config = yaml.safe_load(f)

pd.set_option('display.max_columns', None)
print("✓ Imports and configs loaded successfully")


### File Definitions

- **user_info_df**: DataFrame of respondent x task level data for all users (not just wonky studies)
- **wonky_studies_df**: DataFrame of respondents involved in studies with unexpected outcomes (negative impacts when positive expected)

A study is "wonky" if the outcome is unexpected (e.g., advertisement showed negative impacts of media, which is counter-intuitive).


### Load

In [0]:
# Load data directly from misc folder
notebook_path = os.getcwd() 
repo_root = os.path.abspath(os.path.join(notebook_path, ".."))
misc_dir = os.path.join(repo_root, "misc")

# Direct paths to CSV files in misc folder
user_info_df = pd.read_csv(os.path.join(misc_dir, "user_info_df.csv"))
wonky_counts = pd.read_csv(os.path.join(misc_dir, "wonky_user_counts.csv"))
wonky_respondent_df = pd.read_csv(os.path.join(misc_dir, "wonky_respondent_df.csv"))
wonky_respondent_summary = pd.read_csv(os.path.join(misc_dir, "wonky_respondent_summary.csv"))

# Parse date columns if they exist (with error handling)
if 'date_completed' in user_info_df.columns:
    user_info_df['date_completed'] = pd.to_datetime(user_info_df['date_completed'], errors='coerce')
    print(f"✓ Parsed date_completed column: {user_info_df['date_completed'].dtype}")
    if user_info_df['date_completed'].isna().all():
        print("⚠ Warning: All date_completed values are NaT. Check date format.")
else:
    print("⚠ Warning: 'date_completed' column not found in user_info_df")
    print(f"Available columns: {list(user_info_df.columns)[:10]}...")

if 'first_task_date' in wonky_respondent_summary.columns:
    wonky_respondent_summary['first_task_date'] = pd.to_datetime(wonky_respondent_summary['first_task_date'], errors='coerce')
if 'last_task_date' in wonky_respondent_summary.columns:
    wonky_respondent_summary['last_task_date'] = pd.to_datetime(wonky_respondent_summary['last_task_date'], errors='coerce')

print(f"\n✓ Loaded data from {misc_dir}")
print(f"  - user_info_df: {len(user_info_df):,} rows, {len(user_info_df.columns)} columns")
print(f"  - wonky_counts: {len(wonky_counts):,} rows")
print(f"  - wonky_respondent_df: {len(wonky_respondent_df):,} rows")
print(f"  - wonky_respondent_summary: {len(wonky_respondent_summary):,} rows")


In [0]:
wonky_respondent_df

In [0]:
print(user_info_df.head())

print(wonky_respondent_df.head())

df = pd.DataFrame(user_info_df.isnull().sum(), columns=['null_count'])
display(df.reset_index())

print("\nwonky_studies_df - Missing values:")
missing_wonky = wonky_respondent_df.isnull().sum()
print(missing_wonky[missing_wonky > 0])

In [0]:
wonky_counts

In [0]:
key_numeric_cols = ['task_time_taken_s', 'task_points', 'risk', 'quality', 'task_completed']
available_cols = [col for col in key_numeric_cols if col in user_info_df.columns]
print(user_info_df[available_cols].describe())

if 'wonky_study_flag' in user_info_df.columns:
    print("\n" + "=" * 80)
    print("COMPARISON BY wonky_study_flag (Task Level)")
    print("=" * 80)
    comparison_cols = ['task_time_taken_s', 'task_points', 'risk', 'quality']
    comparison_cols = [col for col in comparison_cols if col in user_info_df.columns]
    
    if len(comparison_cols) > 0:
        wonky_study_tasks = user_info_df[user_info_df['wonky_study_flag'] == 1]
        non_wonky_study_tasks = user_info_df[user_info_df['wonky_study_flag'] == 0]
        
        print("\nWonky Study Tasks (wonky_study_flag=1):")
        print(wonky_study_tasks[comparison_cols].describe())
        
        print("\nNon-Wonky Study Tasks (wonky_study_flag=0):")
        print(non_wonky_study_tasks[comparison_cols].describe())
        
        if 'wonky_studies_count' in user_info_df.columns:
            wonky_user_tasks = user_info_df[user_info_df['wonky_studies_count'] > 0]
            print("\nTasks from Users with Wonky Studies (wonky_studies_count > 0):")
            print(wonky_user_tasks[comparison_cols].describe())

print("\n" + "=" * 80)
print("STATISTICAL SUMMARY: wonky_studies_df")
print("=" * 80)
print(wonky_counts.describe())


### Feature Engineering

In [0]:
# Create time features using modular function
# Verify date column exists and is datetime type
if 'date_completed' not in user_info_df.columns:
    raise ValueError("Column 'date_completed' not found in user_info_df. Available columns: " + str(list(user_info_df.columns)[:20]))

if not pd.api.types.is_datetime64_any_dtype(user_info_df['date_completed']):
    print("⚠ Warning: date_completed is not datetime type. Attempting to convert...")
    print(f"Current dtype: {user_info_df['date_completed'].dtype}")
    print(f"Sample values: {user_info_df['date_completed'].head(3).tolist()}")
    user_info_df['date_completed'] = pd.to_datetime(user_info_df['date_completed'], errors='coerce')
    if user_info_df['date_completed'].isna().all():
        raise ValueError("Failed to parse date_completed column. All values are NaT.")
    print(f"✓ Converted to datetime: {user_info_df['date_completed'].dtype}")

user_info_df = create_time_features(user_info_df, date_col="date_completed")

print(f"Night tasks: {user_info_df['is_night'].mean()*100:.1f}%")
print(f"Weekend tasks: {user_info_df['is_weekend'].mean()*100:.1f}%")


### Behavioural Stuff

#### Temporal Feature Analysis & Breakdowns - STRONG HYPOTHESIS

Analyzing temporal patterns to identify differences between wonky and non-wonky study tasks.


In [0]:
user_info_df['wonky_task_instances'].unique()

In [0]:
temporal_features = [
    "is_weekday",
    "is_weekend",
    "is_night",
    "is_business_hour",
    "is_business_hour_weekday",
    "is_business_hour_weekend",
    "is_monday",
    "is_tuesday",
    "is_wednesday",
    "is_thursday",
    "is_friday",
    "is_saturday",
    "is_sunday",
]

print(
    create_temporal_breakdown_summary(
        user_info_df,
        temporal_features=temporal_features,
        group_col="wonky_task_instances",
        group_threshold=0,
    )
)

**Task complete time - definately good gauge. Majority takes place during business hours, relatively evenly spread across the work week LARGEST detla where wonky is more prevalent is in business hours suggesting professional behaviours
**

##### Chi-Squared Tests for Temporal Features

Testing independence between temporal features and wonky study participation.
Chi-squared test determines if temporal patterns differ significantly between wonky and non-wonky groups.


In [0]:
chi_square_results = perform_chi_square_tests(
    user_info_df,
    feature_set=temporal_features,
    group_var='wonky_task_instances',
    significance_level=0.01
)

chi_square_results

**Strong significance for all subsets but wednesday and friday although this doesn't taken into consideration directionality -> simple separators are business hours**

STRONG SIGNIFICANT READ AT 99% LEVEL LARGEST MAGNITUDE FOUND AT NIGHT. LOWEST MAGNITUDE DURING WEEKEND

In [0]:
# Summary
if len(chi_square_results) > 0:
    significant_features = chi_square_results[chi_square_results['significant'] == True]
    print(f"\nSignificant temporal features (p < 0.01): {len(significant_features)}")
    if len(significant_features) > 0:
        print("Features:", ', '.join(significant_features.index.tolist()))

In [0]:
# Visualize chi-squared statistics
if len(chi_square_results) > 0:
    fig_chi2 = create_chi_squared_bar_chart(
        chi_square_results,
        chi2_col='chi2',
        p_value_col='chi_p_value',
        significance_level=0.01,
        title="Chi-Squared Statistic by Temporal Feature"
    )
    fig_chi2.show()
else:
    print("No chi-squared test results available for visualization")


In [0]:
if len(chi_square_results) > 0:
    delta_results = calculate_temporal_feature_deltas(
      user_info_df,
      temporal_features=temporal_features,
      group_col='wonky_task_instances',  
      group_threshold=0 
)
    
    if len(delta_results) > 0:
        fig_dual = create_chi_squared_delta_dual_axis_chart(
            chi_square_results,
            delta_results,
            chi2_col='chi2',
            p_value_col='chi_p_value',
            delta_col='delta_pct',
            significance_level=0.01,
            title="Chi-Squared Statistic and Delta % by Temporal Feature"
        )
        fig_dual.show()
    else:
        print("No delta results available for visualization")
else:
    print("No chi-squared test results available for visualization")

**Bar is the level of siginficance between the wonky and non wonky, the line are the delta's between wonky and non wonky in terms of when tasks are complete.

positive delta means wonky participants are more prevalent and negative delta means they are less prevalent.

Business hours, Night time, Saturdays look like the overall best separators between wonky and non wonky participants in terms of task complete time
**

#### Task speed features - OKAY HYPOTHESIS

In [0]:
user_info_df['task_time_taken_s_capped'] = np.where(user_info_df['task_time_taken_s'] < user_info_df['task_time_taken_s'].quantile(0.9999), user_info_df['task_time_taken_s'], user_info_df['task_time_taken_s'].quantile(0.9999))

In [0]:
user_info_df = create_task_speed_features(
    user_info_df,
    task_time_col="task_time_taken_s_capped",
    use_std_dev=True
)

mean_time = user_info_df["task_time_taken_s_capped"].mean()
std_time = user_info_df["task_time_taken_s_capped"].std()
print(f"Task time statistics:")
print(f"  Mean: {mean_time:.2f} seconds ({mean_time/60:.2f} minutes)")
print(f"  Std Dev: {std_time:.2f} seconds ({std_time/60:.2f} minutes)")
print(f"  Fast threshold (mean - 1σ): {mean_time - std_time:.2f}s")
print(f"  Suspiciously fast threshold (mean - 2σ): {mean_time - 2*std_time:.2f}s")
print(f"  Slow threshold (mean + 1σ): {mean_time + std_time:.2f}s")
print(f"  Suspiciously slow threshold (mean + 2σ): {mean_time + 2*std_time:.2f}s")
print()

# Display breakdown with wonky vs non-wonky comparison
print(create_task_speed_breakdown_summary(
    user_info_df,
    group_col='wonky_task_instances',
    group_threshold=0
))

**Wonky participants are usually suspcisouly fast to normal non wonky participants tend to be normal to supcisouly slow in terms of delta **

In [0]:
user_info_df = create_task_speed_features(
    user_info_df,
    task_time_col="task_time_taken_s",
    use_std_dev=True
)

fast_threshold = user_info_df["task_time_taken_s"].quantile(0.16)
suspiciously_fast_threshold = user_info_df["task_time_taken_s"].quantile(0.025)
slow_threshold = user_info_df["task_time_taken_s"].quantile(0.84)
suspiciously_slow_threshold = user_info_df["task_time_taken_s"].quantile(0.975)

# Also calculate trimmed mean/std for reference (trimming extreme outliers)
trimmed_data = user_info_df["task_time_taken_s"].clip(
    lower=user_info_df["task_time_taken_s"].quantile(0.01),
    upper=user_info_df["task_time_taken_s"].quantile(0.99)
)
mean_time = trimmed_data.mean()
std_time = trimmed_data.std()

print(f"Task time statistics (using percentiles, robust to outliers):")
print(f"  Mean (trimmed 1%-99%): {mean_time:.2f} seconds ({mean_time/60:.2f} minutes)")
print(f"  Std Dev (trimmed 1%-99%): {std_time:.2f} seconds ({std_time/60:.2f} minutes)")
print(f"  Fast threshold (16th percentile): {fast_threshold:.2f}s ({fast_threshold/60:.2f} min)")
print(f"  Suspiciously fast threshold (2.5th percentile): {suspiciously_fast_threshold:.2f}s ({suspiciously_fast_threshold/60:.2f} min)")
print(f"  Slow threshold (84th percentile): {slow_threshold:.2f}s ({slow_threshold/60:.2f} min)")
print(f"  Suspiciously slow threshold (97.5th percentile): {suspiciously_slow_threshold:.2f}s ({suspiciously_slow_threshold/60:.2f} min)")
print()

# Display breakdown with wonky vs non-wonky comparison
group_col_to_use = 'wonky_task_instances' if 'wonky_task_instances' in user_info_df.columns else 'wonky_study_count'
print(create_task_speed_breakdown_summary(
    user_info_df,
    group_col=group_col_to_use,
    group_threshold=0
))

In [0]:
from eda.statistical_tests import compare_speed_categories_proportions

# Statistical tests for speed categories: Chi-squared and Two-Proportion Z-Test
speed_features = ['is_suspiciously_fast', 'is_fast', 'is_normal_speed', 'is_slow', 'is_suspiciously_slow']

# Ensure we have the correct group column
group_col_to_use = 'wonky_task_instances' if 'wonky_task_instances' in user_info_df.columns else 'wonky_study_count'

print("=" * 80)
print("STATISTICAL TESTS: Speed Categories vs Wonky Status")
print("=" * 80)
print()

# 1. Chi-squared test (overall association)
print("1. CHI-SQUARED TEST (Overall Association)")
print("-" * 80)
print("Tests if speed category distribution is independent of wonky task status")
print()

chi2_speed_results = perform_chi_square_tests(
    user_info_df,
    feature_set=speed_features,
    group_var=group_col_to_use,
    significance_level=0.01
)

if len(chi2_speed_results) > 0:
    print("Results:")
    display(chi2_speed_results)
    print()
    
    significant_features = chi2_speed_results[chi2_speed_results['significant']]
    if len(significant_features) > 0:
        print(f"✓ Significant association found for {len(significant_features)} speed category(ies)")
        print(f"  Features: {', '.join(significant_features.index.tolist())}")
    else:
        print("✗ No significant association found (p > 0.05)")
else:
    print("No results available (check if speed features exist in DataFrame)")
print()

# 2. Two-proportion Z-test (individual category comparisons)
print("2. TWO-PROPORTION Z-TEST (Individual Category Comparisons)")
print("-" * 80)
print("Tests if proportion differs significantly for each speed category")
print()

ztest_speed_results = compare_speed_categories_proportions(
    user_info_df,
    speed_features=speed_features,
    group_col=group_col_to_use,
    group_threshold=0,
    significance_level=0.01
)

if len(ztest_speed_results) > 0:
    print("Results:")
    display(ztest_speed_results.reset_index(drop=True))
    print()
    
    # Format results for easier interpretation
    print("Summary:")
    for _, row in ztest_speed_results.iterrows():
        feature_name = row['feature'].replace('is_', '').replace('_', ' ').title()
        sig_marker = "***" if row['significant'] else ""
        print(f"  {feature_name}:")
        print(f"    Wonky: {row['wonky_proportion']*100:.2f}% ({row['wonky_count']:,}/{row['wonky_total']:,})")
        print(f"    Non-wonky: {row['non_wonky_proportion']*100:.2f}% ({row['non_wonky_count']:,}/{row['non_wonky_total']:,})")
        print(f"    Difference: {row['proportion_diff']*100:+.2f}%")
        print(f"    Z-statistic: {row['z_statistic']:.3f}, p-value: {row['p_value']:.4f} {sig_marker}")
        print()
    
    significant_categories = ztest_speed_results[ztest_speed_results['significant']]
    if len(significant_categories) > 0:
        print(f"✓ Significant differences found for {len(significant_categories)} category(ies):")
        for _, row in significant_categories.iterrows():
            feature_name = row['feature'].replace('is_', '').replace('_', ' ').title()
            direction = "higher" if row['proportion_diff'] > 0 else "lower"
            print(f"  - {feature_name}: Wonky tasks have {abs(row['proportion_diff']*100):.2f}% {direction} proportion")
    else:
        print("✗ No significant differences found (p > 0.05 for all categories)")
else:
    print("No results available (check if speed features exist in DataFrame)")
print()

print("=" * 80)
print("Interpretation:")
print("- Chi-squared test: Overall test of independence (all categories together)")
print("- Two-proportion Z-test: Individual tests for each category")
print("- Significant results (p < 0.01) indicate wonky and non-wonky groups differ")
print("=" * 80)

TODO -> rethink Viz for this section all significant across z test and chi squared test

#### Task and Point categroy - WEAK HYPOTHESIS

In [0]:
user_info_df['defined_task_category'] = user_info_df['taskCategory'].astype(str) + "_points_" + user_info_df['payoutPoints'].astype(str)

In [0]:
user_info_df = user_info_df.merge(wonky_respondent_df[['balance_respondentPk', 'task_pk', 'wonky_study_count']], left_on=['balance_respondentPk', 'taskPk'], right_on=['balance_respondentPk', 'task_pk'], how='left')

In [0]:
# Import the function
from eda.statistical_tests import compare_task_category_wonky_rates

# Create the summary table directly from user_info_df
# Why: Analyzes wonky rates by task category to identify which categories have higher fraud rates
category_wonky_summary = compare_task_category_wonky_rates(
    df=user_info_df,
    category_col="defined_task_category",
    respondent_id_col="respondentPk",
    wonky_col="wonky_study_count"
)

# Display the results
print("Wonky Rates by Task Category:")
print("=" * 100)
display(category_wonky_summary)

# Format for better readability
category_wonky_summary_formatted = category_wonky_summary.copy()
category_wonky_summary_formatted['wonky_pct'] = category_wonky_summary_formatted['wonky_pct'].round(2)
category_wonky_summary_formatted['non_wonky_pct'] = category_wonky_summary_formatted['non_wonky_pct'].round(2)
category_wonky_summary_formatted['proportion_delta'] = category_wonky_summary_formatted['proportion_delta'].round(2)

display(category_wonky_summary_formatted)

In [0]:
category_wonky_summary

#### Device & Platform

In [0]:
sorted(user_info_df.columns)

In [0]:
user_info_df['hardware_version2'] = user_info_df['hardware_version'].fillna('unknown')

In [0]:
a = user_info_df[
    ["respondentPk", "hardware_version", "platform_name", "wonky_study_count", "wonky_task_instances"]
].groupby(["hardware_version", "platform_name"]).agg({'wonky_study_count': 'sum',
                                                      'respondentPk': 'count'})

a['wonky_rate'] = a['wonky_study_count'] / a['respondentPk']

a

### Demographic stuff

In [0]:
# user_info_df_shortened = user_info_df[['respondentPk', 'taskPk', 'device', 'employer_annual_revenue', 'fulcrum_employment_status', 'fulcrum_household_income', 'gambling_participation_mc', 'gender', 'home_owner', 'wonky_study_count', 'wonky_task_instances']]

In [0]:
series = user_info_df['gambling_participation_mc']

# One-hot encode each gambling mode
gambling_dummies = (
    series.explode()                    
     .str.strip()                   
     .pipe(pd.get_dummies)          
     .groupby(level=0).sum()     
)

gambling_cols = gambling_dummies.columns

user_info_df = user_info_df.join(gambling_dummies)

In [0]:
gambling_cols

In [0]:
income_map = {
    "A": "Less than £15,000",
    "B": "£15,000 to £19,999",
    "C": "£20,000 to £24,999",
    "D": "£25,000 to £29,999",
    "E": "£30,000 to £34,999",
    "F": "£35,000 to £39,999",
    "G": "£40,000 to £44,999",
    "H": "£45,000 to £49,999",
    "I": "£50,000 to £59,999",
    "J": "£60,000 to £74,999",
    "K": "£75,000 to £84,999",
    "L": "£85,000 to £99,999",
    "M": "£100,000 to £124,999",
    "N": "£125,000 to £149,999",
    "O": "£150,000 to £174,999",
    "P": "£175,000 to £199,999",
    "Q": "£200,000 and above",
    "R": "Prefer not to answer",
}

user_info_df["fulcrum_household_income_mapped"] = (
    user_info_df["fulcrum_household_income"].map(income_map)
)

user_info_df["fulcrum_household_income_mapped"].value_counts()

In [0]:
demographic_cols = [
    "platform_name",
    "hardware_version",
    "survey_locale",
    "gender",
    "fulcrum_household_income_mapped",
    "device",
    "home_owner",
] + list(gambling_cols)

In [0]:
# Create respondent-level behavioral features
import importlib

if "eda.feature_engineering" in sys.modules:
    importlib.reload(sys.modules["eda.feature_engineering"])
    from eda.feature_engineering import create_respondent_behavioral_features

respondent_features = create_respondent_behavioral_features(
    user_info_df,
    respondent_id_col="respondentPk",
    date_col="date_completed",
    config={
        "high_volume_percentile": feature_config["volume_thresholds"][
            "high_volume_percentile"
        ],
        "extreme_volume_percentile": feature_config["volume_thresholds"][
            "extreme_volume_percentile"
        ],
        "velocity_bins": feature_config["velocity_bins"],
        "velocity_labels": feature_config["velocity_labels"],
    },
    demographic_cols= demographic_cols,
)

print(f"Aggregated to {respondent_features.shape[0]:,} respondents")
print(f"Avg tasks per respondent: {respondent_features['total_tasks'].mean():.2f}")
print(
    f"Avg suspicious fast rate: {respondent_features['suspicious_fast_rate'].mean()*100:.2f}%"
)

In [0]:
respondent_features

In [0]:
# Add wonky features using modular function
respondent_features = add_wonky_features(
    respondent_features, wonky_counts, respondent_id_col="respondentPk"
)

print(f"Wonky features added")
print(
    f"Respondents with wonky tasks: {(respondent_features['wonky_task_ratio'] > 0).sum():,}"
)
print(
    f"High wonky concentration (>50%): {respondent_features['is_high_wonky'].sum():,}"
)

respondent_features.head(100)

respondent_features.head(100)

In [ ]:
# Create binary target variables for wonky study detection
# has_wonky_tasks: based on wonky_task_instances (any wonky task involvement)
# has_wonky_studies: based on wonky_study_count (task-specific wonky studies)

# Fill NaN values in wonky columns with 0 for binary classification
if 'wonky_task_instances' in respondent_features.columns:
    respondent_features['wonky_task_instances'] = respondent_features['wonky_task_instances'].fillna(0)
if 'wonky_study_count' in respondent_features.columns:
    respondent_features['wonky_study_count'] = respondent_features['wonky_study_count'].fillna(0)
if 'wonky_task_ratio' in respondent_features.columns:
    respondent_features['wonky_task_ratio'] = respondent_features['wonky_task_ratio'].fillna(0)

# Create binary flags
respondent_features['has_wonky_tasks'] = (respondent_features['wonky_task_instances'] > 0).astype(int)
respondent_features['has_wonky_studies'] = (respondent_features['wonky_study_count'] > 0).astype(int)

# Create severity tiers based on wonky_task_ratio
respondent_features['wonky_severity'] = pd.cut(
    respondent_features['wonky_task_ratio'],
    bins=[-0.01, 0.01, 0.2, 0.5, 1.0],
    labels=['None', 'Low', 'Medium', 'High'],
    include_lowest=True
)
respondent_features['wonky_severity'] = respondent_features['wonky_severity'].fillna('None')

# Check class balance
print("=" * 80)
print("TARGET VARIABLE DISTRIBUTION")
print("=" * 80)
print(f"\nTotal respondents: {len(respondent_features):,}")

print(f"\nhas_wonky_tasks distribution:")
wonky_tasks_dist = respondent_features['has_wonky_tasks'].value_counts().sort_index()
for val, count in wonky_tasks_dist.items():
    pct = (count / len(respondent_features)) * 100
    label = "Has wonky tasks" if val == 1 else "No wonky tasks"
    print(f"  {label}: {count:,} ({pct:.2f}%)")

print(f"\nhas_wonky_studies distribution:")
wonky_studies_dist = respondent_features['has_wonky_studies'].value_counts().sort_index()
for val, count in wonky_studies_dist.items():
    pct = (count / len(respondent_features)) * 100
    label = "Has wonky studies" if val == 1 else "No wonky studies"
    print(f"  {label}: {count:,} ({pct:.2f}%)")

print(f"\nWonky severity distribution:")
severity_dist = respondent_features['wonky_severity'].value_counts()
for severity, count in severity_dist.items():
    pct = (count / len(respondent_features)) * 100
    print(f"  {severity}: {count:,} ({pct:.2f}%)")

# Check data quality
print(f"\n{'=' * 80}")
print("DATA QUALITY CHECK")
print("=" * 80)
print(f"\nMissing values per column:")
missing = respondent_features.isnull().sum()
missing_pct = (missing / len(respondent_features)) * 100
missing_df = pd.DataFrame({
    'missing_count': missing,
    'missing_pct': missing_pct
})
missing_df = missing_df[missing_df['missing_count'] > 0].sort_values('missing_count', ascending=False)
if len(missing_df) > 0:
    display(missing_df)
else:
    print("  No missing values found!")

print(f"\nData shape: {respondent_features.shape}")
print(f"Total columns: {len(respondent_features.columns)}")

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

respondent_features.head(100)

In [ ]:
respondent_features.head(100)

In [0]:
respondent_features.head(100)

In [0]:
user_info_df[user_info_df['respondentPk'] == 'e043970a-f4a2-4a33-afdd-eccb63ec8271']

### Inconsitency flags

In [0]:
# Compare wonky task rates across demographic groups

demographic_cols = ['platform_name', 'hardware_version', 'survey_locale']

demographic_results = {}

for demo_col in demographic_cols:
    if demo_col in respondent_features.columns:
        print(demo_col)
        print(f"\n{'=' * 80}")
        print(f"Demographic Comparison: {demo_col}")
        print('=' * 80)
        
        # Perform comparisons
        demo_comparisons = compare_demographic_groups(
            respondent_features,
            demographic_col=demo_col,
            target_col='wonky_task_ratio',
            min_group_size=10,
            significance_level=0.05
        )
        
        if len(demo_comparisons) > 0:
            demographic_results[demo_col] = demo_comparisons
            
            # Show summary statistics by group
            group_summary = respondent_features.groupby(demo_col).agg({
                'wonky_task_ratio': ['mean', 'median', 'count'],
                'has_wonky_tasks': 'sum'
            }).round(4)
            group_summary.columns = ['mean_wonky_ratio', 'median_wonky_ratio', 'total_count', 'wonky_count']
            group_summary['wonky_rate'] = (group_summary['wonky_count'] / group_summary['total_count'] * 100).round(2)
            group_summary = group_summary.sort_values('mean_wonky_ratio', ascending=False)
            
            print(f"\nSummary by {demo_col}:")
            display(group_summary)
            
            # Show significant comparisons
            significant_comps = demo_comparisons[demo_comparisons['mw_significant'] == True]
            if len(significant_comps) > 0:
                print(f"\nSignificant differences (p < 0.05): {len(significant_comps)}")
                display(significant_comps[['group1', 'group2', 'mean_difference', 'mw_p_value', 'welch_p_value', 'tests_agree']])
            else:
                print("\nNo significant differences found between groups")
        else:
            print(f"Insufficient data for {demo_col} comparisons")


In [0]:
# # Visualize demographic comparisons
# # Why: Bar charts make it easy to see which demographic groups have higher wonky task rates
# # Helps identify patterns that may inform modeling or business decisions

# for demo_col in demographic_cols:
#     if demo_col in respondent_features.columns and demo_col in demographic_results:
#         # Create summary by group
#         demo_summary = respondent_features.groupby(demo_col).agg({
#             'wonky_task_ratio': 'mean',
#             'has_wonky_tasks': ['sum', 'count']
#         }).reset_index()
#         demo_summary.columns = [demo_col, 'mean_wonky_ratio', 'wonky_count', 'total_count']
#         demo_summary['wonky_rate'] = (demo_summary['wonky_count'] / demo_summary['total_count'] * 100)
#         demo_summary = demo_summary.sort_values('mean_wonky_ratio', ascending=False)
        
#         # Bar chart of wonky rates by xdemographic group
#         fig_demo = create_bar_plot(
#             demo_summary,
#             x=demo_col,
#             y='wonky_rate',
#             title=f'Wonky Task Rate by {demo_col.replace("_", " ").title()}',
#             labels={demo_col: demo_col.replace("_", " ").title(), 'wonky_rate': 'Wonky Task Rate (%)'},
#             color='wonky_rate',
#             color_continuous_scale='Reds',
#             text='wonky_rate',
#             texttemplate='%{text:.1f}%',
#             textposition='outside',
#             tickangle=45
#         )
#         fig_demo.show()
        
#         # Box plot comparing wonky task ratios across groups
#         fig_demo_box = create_box_plot(
#             respondent_features,
#             x=demo_col,
#             y='wonky_task_ratio',
#             title=f'Wonky Task Ratio Distribution by {demo_col.replace("_", " ").title()}',
#             labels={demo_col: demo_col.replace("_", " ").title(), 'wonky_task_ratio': 'Wonky Task Ratio'},
#             tickangle=45
#         )
#         fig_demo_box.show()


### Feature Selection Guidance

Based on EDA results, identify which features show strongest discrimination and should be prioritized for modeling.


#### Modeling Recommendations

Based on EDA findings, recommendations for feature selection and modeling approach.


In [0]:
# # Generate modeling recommendations
# # Why: Synthesizes EDA findings into actionable recommendations for model development
# # Note: Requires statistical_results to be created first (run the statistical tests cell before this one)

# recommendations = []

# # Check statistical test results
# if 'statistical_results' not in globals():
#     recommendations.append("⚠️  Warning: 'statistical_results' not found. Run the 'Hypothesis Testing' section first.")
# elif len(statistical_results) > 0:
#     significant_features = statistical_results[
#         (statistical_results['mw_significant'] == True) & 
#         (statistical_results.get('welch_significant', pd.Series([True] * len(statistical_results))) == True)
#     ]
    
#     if len(significant_features) > 0:
#         recommendations.append(f"✓ {len(significant_features)} features show significant differences (both tests agree)")
#         recommendations.append(f"  → Prioritize these features: {', '.join(significant_features['metric'].head(5).tolist())}")
    
#     # Check effect sizes
#     if 'mean_difference' in statistical_results.columns:
#         large_effect = statistical_results[statistical_results['mean_difference'].abs() > statistical_results['mean_difference'].abs().quantile(0.75)]
#         if len(large_effect) > 0:
#             recommendations.append(f"✓ {len(large_effect)} features show large effect sizes")
#             recommendations.append(f"  → These features have practical significance: {', '.join(large_effect['metric'].head(3).tolist())}")

# # Check temporal features
# if len(chi_square_results) > 0:
#     sig_temporal = chi_square_results[chi_square_results['significant'] == True]
#     if len(sig_temporal) > 0:
#         recommendations.append(f"✓ {len(sig_temporal)} temporal features are significantly associated with wonky studies")
#         recommendations.append(f"  → Include temporal features: {', '.join(sig_temporal.index.tolist())}")

# # Check demographic differences
# if demographic_results:
#     total_sig_demo = sum(len(df[df['mw_significant'] == True]) for df in demographic_results.values())
#     if total_sig_demo > 0:
#         recommendations.append(f"✓ Found {total_sig_demo} significant demographic group differences")
#         recommendations.append(f"  → Consider including demographic features as interaction terms")

# print("=" * 80)
# print("MODELING RECOMMENDATIONS")
# print("=" * 80)
# print()
# for rec in recommendations:
#     print(rec)
# print()
# print("=" * 80)
# print("NEXT STEPS:")
# print("=" * 80)
# print("1. Use top-ranked features from feature ranking for initial model")
# print("2. Include temporal features that show significant associations")
# print("3. Consider demographic interactions if significant differences found")
# print("4. Validate feature importance using model-based methods (e.g., Random Forest feature importance)")
# print("5. Monitor model performance on features identified as significant in EDA")


In [0]:
# # Create fraud risk score using modular function
# respondent_features = create_fraud_risk_score(
#     respondent_features,
#     config={
#         'fraud_score_weights': feature_config['fraud_score_weights'],
#         'fraud_score_thresholds': feature_config['fraud_score_thresholds'],
#         'fraud_score_bins': feature_config['fraud_score_bins'],
#         'fraud_score_labels': feature_config['fraud_score_labels'],
#         'suspected_fraud_threshold': feature_config['suspected_fraud_threshold']
#     }
# )

# # Create wonky risk score
# respondent_features = create_wonky_risk_score(
#     respondent_features,
#     config={
#         'wonky_score_weights': feature_config['wonky_score_weights'],
#         'wonky_score_thresholds': feature_config['wonky_score_thresholds'],
#         'wonky_score_bins': feature_config['wonky_score_bins'],
#         'wonky_score_labels': feature_config['wonky_score_labels']
#     }
# )

# print("Fraud risk distribution:")
# print(respondent_features['fraud_risk_tier'].value_counts().sort_index())
# print(f"Suspected fraud rate: {respondent_features['suspected_fraud'].mean()*100:.2f}%")


### Hypothesis Testing

Running Mann-Whitney U test and Welch's t-test for validation.

Check if Mann-Whitney U and Welch's t-test agree on significance.
When both tests agree, we have higher confidence in the results.
When they disagree, it signals need for further investigation.

Compare wonky vs non-wonky users using statistical tests.
- Mann-Whitney U test (non-parametric, primary test)
- Welch's t-test (parametric validator/sense check)
Uses `compare_groups_with_both_tests()` to run both tests simultaneously.

In [0]:
# # Create binary flag for wonky users
# respondent_features['has_wonky_tasks'] = (respondent_features['wonky_task_ratio'] > 0).astype(int)

# print(f"Wonky users: {respondent_features['has_wonky_tasks'].sum():,} ({respondent_features['has_wonky_tasks'].mean()*100:.2f}%)")
# print(f"Non-wonky users: {(respondent_features['has_wonky_tasks'] == 0).sum():,} ({(respondent_features['has_wonky_tasks'] == 0).mean()*100:.2f}%)")

### Statistical Test Results Visualization

Visualizing statistical test results to identify features with strongest discrimination between wonky and non-wonky groups.

In [0]:
# # Perform statistical tests using modular function
# statistical_results = compare_groups_with_both_tests(
#     respondent_features,
#     group_col=stats_config['group_comparison']['group_col'],
#     metrics=stats_config['test_metrics'],
#     group1_value=stats_config['group_comparison']['group1_value'],
#     group2_value=stats_config['group_comparison']['group2_value'],
#     significance_level=stats_config['significance_level']
# )

# print("Statistical Test Results (Mann-Whitney U + Welch's t-test):")
# print("=" * 100)
# display(statistical_results)


In [0]:
# # Create dual-axis chart showing count differences and Welch's t-statistic

# # Check if statistical_results exists
# if 'statistical_results' not in globals():
#     print("Warning: 'statistical_results' not found.")
#     print("   Please run the 'Hypothesis Testing' section cells first to create statistical_results.")
#     print("   This visualization will be skipped.")
# elif len(statistical_results) > 0 and 'welch_statistic' in statistical_results.columns:
#     # Calc normalized count differences
#     testdf_2 = statistical_results.copy()
#     testdf_2['count_difference'] = testdf_2['wonky_mean'] - testdf_2['non_wonky_mean']
#     testdf_2['count_difference_nrm'] = testdf_2['count_difference'] / (testdf_2['non_wonky_mean'].replace(0, np.nan))
    
#     testdf_2 = testdf_2.set_index('metric')
    
#     fig_dual = create_dual_axis_statistical_chart(
#         testdf_2,
#         count_diff_col='count_difference_nrm',
#         t_stat_col='welch_statistic',
#         title="Ave Wonky Count Difference & Welch's t-statistic by Feature"
#     )
#     fig_dual.show()
# else:
#     print("Statistical results not available for dual-axis chart")
#     if 'statistical_results' in globals():
#         print(f"   statistical_results exists but is empty or missing 'welch_statistic' column")
#         print(f"   Available columns: {list(statistical_results.columns) if len(statistical_results) > 0 else 'N/A'}")


In [0]:
# # Analyze thresholds for key features
# print("Threshold Analysis: Task Volume")
# print("=" * 80)
# volume_thresholds = analyze_thresholds(
#     respondent_features,
#     feature='total_tasks',
#     bins=stats_config['threshold_analysis']['default_bins'],
#     target_col='has_wonky_tasks'
# )
# display(volume_thresholds)

# print("\nThreshold Analysis: Suspicious Fast Rate")
# print("=" * 80)
# speed_thresholds = analyze_thresholds(
#     respondent_features,
#     feature='suspicious_fast_rate',
#     bins=stats_config['threshold_analysis']['default_bins'],
#     target_col='has_wonky_tasks'
# )
# display(speed_thresholds)

In [0]:
# # Example visualizations using modular functions

# # Histogram of fraud risk score
# fig_fraud_risk = create_histogram(
#     respondent_features,
#     x='fraud_risk_score',
#     color='fraud_risk_tier',
#     nbins=11,
#     title='Fraud Risk Score Distribution',
#     labels={'fraud_risk_score': 'Fraud Risk Score (0-10)'}
# )
# fig_fraud_risk.show()

# # Box plot comparing wonky vs non-wonky users
# fig_speed = create_box_plot(
#     respondent_features,
#     x='has_wonky_tasks',
#     y='avg_task_time',
#     color='has_wonky_tasks',
#     title='Average Task Time: Wonky vs Non-Wonky Users',
#     labels={'has_wonky_tasks': 'Has Wonky Tasks (1=Yes, 0=No)', 'avg_task_time': 'Average Task Time (seconds)'},
#     category_orders={'has_wonky_tasks': [0, 1]},
#     boxmean='sd'
# )
# fig_speed.show()

## Summary

**Key Findings:**
- Statistical tests (Mann-Whitney U + Welch's t-test) identify significant differences between wonky and non-wonky users
- Temporal features show strong associations with wonky study participation (chi-squared tests)
- Feature engineering creates actionable behavioral indicators (task speed, volume, velocity)
- Demographic groups (platforms, hardware, locales) show varying wonky task rates
- Both statistical tests provide validation - agreement increases confidence in findings
- Visualizations highlight features with strongest discrimination power

**Feature Selection Insights:**
- Top-ranked features combine statistical significance with practical effect sizes
- Temporal patterns (business hours, night tasks) are strong discriminators
- Behavioral features (task volume, speed) show consistent differences
- Demographic features may be useful as interaction terms

**Next Steps:**
- Use top-ranked features from feature ranking for model training
- Include temporal features that show significant chi-squared associations
- Consider demographic interactions based on group comparison results
- Apply fraud risk thresholds to filter suspicious users
- Build models using features identified as significant in EDA
- Validate feature importance using model-based methods (Random Forest feature importance)
